# Restroom Redoubt

In [4]:
import re

## Part 1

In [1]:
def parse_input(sample=True, file_path="Restroom_Redoubt.txt", mode="r"):
	if sample:
		file_path = "sample.txt"
	
	with open(file_path, mode) as file:
		data = file.read().splitlines()

	return data

In [10]:
parse_input()

['p=0,4 v=3,-3',
 'p=6,3 v=-1,-3',
 'p=10,3 v=-1,2',
 'p=2,0 v=2,-1',
 'p=0,0 v=1,3',
 'p=3,0 v=-2,-2',
 'p=7,6 v=-1,-3',
 'p=3,0 v=-1,-2',
 'p=9,3 v=2,3',
 'p=7,3 v=-1,2',
 'p=2,4 v=2,-3',
 'p=9,5 v=-3,-3']

In [13]:
def extract_pos_vel(data):
	robots = []
	pattern = (
		r"p=(-?\d+),(-?\d+) v=(-?\d+),(-?\d+)"
	)
	for line in data:
		current_bot = []
		match = re.match(pattern, line)
		if match:
			current_bot.append([int(x) for x in match.groups()])

		robots.append(current_bot)

	return robots

In [ ]:
def simulate_movement(robots, seconds=100, tiles_dim=(11, 7)):
	pass